# Batch Reinforcement Learning with Coach

Many real-world problems do not have a simulator modelling the environment, that the agent would be interacting with, in standard Reinforcement Learning. Instead, all we have is some data that was collected using some deployed policy, and we would like to use that data in order to learn a better policy for solving the problem. 
One such example might be developing a better drug dose and/or admission schedule policy. We have data based on the policy that was used with patients so far, but cannot experiment (and explore) on patients to collect new data. 

But, wait... If we don't have a simulator, how would we evaluate our newly learned policy and know if it is any good? Which algorithms should we be using in order to better address the problem of learning from a batch of data? Alternatively, what do we do if we don't have a simulator, but instead can actually deploy our policy on that real-world problem, and would just like to separate the new data collection from the learning a new policy part (i.e. if we have a system that can quite easily run inference, but is very hard to integrate a reinforcement learning framework with, such as Coach, for learning a new policy. **Can we have an actual example here? i.e. something like an oil company with nodes that can only do inference**), and iteratively train a better and better policy? 

We will try to address these questions and more in this tutorial, demonstrating how to use batch reinforcement learning. 

First, let's use some simple environment to collect the data to be used for learning an agent using BatchRL. In reality, we probably would already have a dataset of transitions of the form `<current_observation, action, reward, next_state>` to be used for learning a new agent. Ideally, we would also have, for each transtion, p(a|o) the probabilty of taking that action, given that transition's `current_observation`. 

## Preliminaries
First, get the required imports and other general settings going for this notebook.

In [12]:
from copy import deepcopy
import tensorflow as tf
import os

from rl_coach.agents.dqn_agent import DQNAgentParameters
from rl_coach.agents.ddqn_bcq_agent import DDQNBCQAgentParameters, KNNParameters
from rl_coach.base_parameters import VisualizationParameters
from rl_coach.core_types import TrainingSteps, EnvironmentEpisodes, EnvironmentSteps, CsvDataset
from rl_coach.environments.gym_environment import GymVectorEnvironment
from rl_coach.graph_managers.batch_rl_graph_manager import BatchRLGraphManager
from rl_coach.graph_managers.graph_manager import ScheduleParameters
from rl_coach.memories.memory import MemoryGranularity
from rl_coach.schedules import LinearSchedule
from rl_coach.memories.episodic import EpisodicExperienceReplayParameters
from rl_coach.architectures.head_parameters import QHeadParameters
from rl_coach.agents.ddqn_agent import DDQNAgentParameters
from rl_coach.base_parameters import TaskParameters
from rl_coach.spaces import SpacesDefinition, DiscreteActionSpace, VectorObservationSpace, StateSpace, RewardSpace

# Get all the outputs of this tutorial out on the 'BatchRL' folder
os.chdir('BatchRL')

# the dataset size to collect 
DATASET_SIZE = 50000

task_parameters = TaskParameters(experiment_path='.')

####################
# Graph Scheduling #
####################

schedule_params = ScheduleParameters()

# 100 epochs (we run train over all the dataset every epoch) of training
schedule_params.improve_steps = TrainingSteps(100)

# we evaluate the model every epoch
schedule_params.steps_between_evaluation_periods = TrainingSteps(1)

schedule_params.evaluation_steps = EnvironmentEpisodes(10)
schedule_params.heatup_steps = EnvironmentSteps(DATASET_SIZE)

################
#  Environment #
################
env_params = GymVectorEnvironment(level='Acrobot-v1')


FileNotFoundError: [Errno 2] No such file or directory: 'BatchRL'

## Solving Acrobot with Batch RL

Let's use Gym's `Acrobot-v1` in order to collect a dataset of experience, and then use that dataset in order to learn an agent solving the environment using Batch RL. 

### The Preset 

First we will collect a dataset using a random action selecting policy. Then we will use that dataset to train an agent in a Batch RL fashion. <br>
Let's start simple - training an agent with Double DQN. 

 

In [3]:
tf.reset_default_graph() # just to clean things up; only needed for the tutorial

#########
# Agent #
#########
agent_params = DDQNAgentParameters()
agent_params.network_wrappers['main'].batch_size = 128
agent_params.algorithm.num_steps_between_copying_online_weights_to_target = TrainingSteps(100)
agent_params.algorithm.discount = 0.99

# to jump start the agent's q values, and speed things up, we'll initialize the last Dense layer
# with something in the order of the discounted reward of a random policy
agent_params.network_wrappers['main'].heads_parameters = \
[QHeadParameters(output_bias_initializer=tf.constant_initializer(-100))]

# NN configuration
agent_params.network_wrappers['main'].learning_rate = 0.0001
agent_params.network_wrappers['main'].replace_mse_with_huber_loss = False

# ER - we'll be needing an episodic replay buffer for off-policy evaluation
agent_params.memory = EpisodicExperienceReplayParameters()

# E-Greedy schedule - there is no exploration in Batch RL. Disabling E-Greedy. 
agent_params.exploration.epsilon_schedule = LinearSchedule(initial_value=0, final_value=0, decay_steps=1)
agent_params.exploration.evaluation_epsilon = 0


graph_manager = BatchRLGraphManager(agent_params=agent_params,
                                    env_params=env_params,
                                    schedule_params=schedule_params,
                                    vis_params=VisualizationParameters(dump_signals_to_csv_every_x_episodes=1),
                                    reward_model_num_epochs=30)
graph_manager.create_graph(task_parameters)
graph_manager.improve()


Creating graph - name: BatchRLGraphManager
Creating agent - name: agent


KeyboardInterrupt: 

Ignoring the prints around off-policy evaluation for now (we'll return to it in a bit), we first see Coach running a long heatup of 50,000 steps (as we have defined a `DATASET_SIZE` of 50,000 in the preliminaries section), in order to collect a dataset of random actions. Then, we can see Coach training a supervised reward model that is needed for OPE (off-policy evaluation). Last, Coach starts using the collected dataset of experience to train a Double DQN agent. Since, for this environment, we actually do have a simulator, Coach will be using it to evaluate the learned policy. As you can probably see, since this is a very simple environment, a dataset of just random actions is enough to get a Double DQN agent training, and reaching rewards of less than -100 (actually solving the environment). As you can also probably notice, the learning is not very stable, and if you'll take a look at the Q values predicted by the agent (e.g. in Coach Dashboard; this tutorial experiment results are under the `tutorials` folder), you will be seeing them unboundedly increasing. This is caused due to the Batch RL based learning where not interacting with the environment any further, while randomly exposing only small parts of the MDP in the dataset, makes learning even harder than standard Off-Policy RL. This phenomena is very nicely explained in [Off-Policy Deep Reinforcement Learning without Exploration](https://arxiv.org/abs/1812.02900). We have implemented a discrete-actions variant of [Batch Constrained Q-Learning](https://github.com/NervanaSystems/coach/blob/master/rl_coach/agents/ddqn_bcq_agent.py), which helps with mitigating this issue. 

Next, let's switch to a dataset containing data combined from several 'deployed' policies, as is often the case in real-world scenarios, where we usually already have some policy (hopefully not a random one) in-place and we want to improve upon. For instance, a recommender system already using some policy for gemerating recommendations, and we want to use Batch RL to acquire a better and improved policy. <br>

We will demonstrate that by training an agent, and using its replay buffer content as the dataset from which we will learn a new agent, based solely on that data (without any further interaction with the environment). This should allow for both a better trained agent and for more meaningful Off-Policy Evaluation (as the more extensive your input data is, i.e. exposing more of the MDP, the better is the evaluation of a new agent based on it. In other words - garbage-in, garbage-out).


In [ ]:
tf.reset_default_graph() # just to clean things up; only needed for the tutorial

# Experience Generating Agent parameters
experience_generating_agent_params = DDQNAgentParameters()

# schedule parameters
experience_generating_schedule_params = ScheduleParameters()
experience_generating_schedule_params.heatup_steps = EnvironmentSteps(1000)
experience_generating_schedule_params.improve_steps = TrainingSteps(
    DATASET_SIZE - experience_generating_schedule_params.heatup_steps.num_steps)
experience_generating_schedule_params.steps_between_evaluation_periods = EnvironmentEpisodes(10)
experience_generating_schedule_params.evaluation_steps = EnvironmentEpisodes(1)

# DQN params
experience_generating_agent_params.algorithm.num_steps_between_copying_online_weights_to_target = EnvironmentSteps(100)
experience_generating_agent_params.algorithm.discount = 0.99
experience_generating_agent_params.algorithm.num_consecutive_playing_steps = EnvironmentSteps(1)

# NN configuration
experience_generating_agent_params.network_wrappers['main'].learning_rate = 0.0001
experience_generating_agent_params.network_wrappers['main'].batch_size = 128
experience_generating_agent_params.network_wrappers['main'].replace_mse_with_huber_loss = False
experience_generating_agent_params.network_wrappers['main'].heads_parameters = \
[QHeadParameters(output_bias_initializer=tf.constant_initializer(-100))]

# ER size
experience_generating_agent_params.memory = EpisodicExperienceReplayParameters()
experience_generating_agent_params.memory.max_size = \
    (MemoryGranularity.Transitions,
     experience_generating_schedule_params.heatup_steps.num_steps +
     experience_generating_schedule_params.improve_steps.num_steps)

# E-Greedy schedule
experience_generating_agent_params.exploration.epsilon_schedule = LinearSchedule(1.0, 0.01, DATASET_SIZE)
experience_generating_agent_params.exploration.evaluation_epsilon = 0

# 50 epochs (we run train over all the dataset every epoch) of training
schedule_params.improve_steps = TrainingSteps(50)

graph_manager = BatchRLGraphManager(agent_params=agent_params,
                                    experience_generating_agent_params=experience_generating_agent_params,
                                    experience_generating_schedule_params=experience_generating_schedule_params,
                                    env_params=env_params,
                                    schedule_params=schedule_params,
                                    vis_params=VisualizationParameters(dump_signals_to_csv_every_x_episodes=1),
                                    reward_model_num_epochs=30,
                                    train_to_eval_ratio=0.5)
graph_manager.create_graph(task_parameters)
graph_manager.improve()


### Off-Policy Evaluation
As we have mentioned earlier, one of the hardest problems in Batch RL is that we do not have a simulator or cannot easily deploy a trained policy on the real-problem system, in order to test its goodness. This is where off-policy evaluation (OPE) comes in handy. </br>

Coach supports several off-policy evaluators, some are aimed at bandits problems (thus the evaluators are only evaluating a single step return. Yet from our experience, there are cases where these also correlate with the return of RL problems), and others are for full-blown Reinforcement Learning problems. The main goal of the OPEs is to help us select the best model to deploy, either for collecting more data to do another round of Batch RL on, or to select our final model for deployment. 

Openning the experiment that we have just ran (under the `tutorials` folder, with Coach Dashboard), you will be able to plot the actual simulator's `Evaluation Reward`. Usually, we won't have this signal available since we won't have a simulator, but since we're using just a dummy environment, for demonstration purposes, we can use it to test for how some of the OPEs correlate with it. 

Here are two example plots from Dashboard showing how well the `Weighted Importance Sampling` (RL estimator) and the `Doubly Robust` (bandits estimator) each correlate with the `Evaluation Reward`. </br>
![Weighted Importance Sampling](BatchRL/img/wis.png "Weighted Importance Sampling vs. Evaluation Reward") 
</br>
![Doubly Robust](BatchRL/img/dr.png "Doubly Robust vs. Evaluation Reward") 



## Using a dataset to feed a Batch RL algorithm 

Ok, so we now understand how things are expected to work. But, hey... if we don't have a simulator (which we did have in this tutorial so far, and have used it to generate a training/evaluation dataset) how will we feed Coach with the dataset to train/evaluate on?

### The CSV
Coach defines an expected csv data, format from which it will read and fill its replay buffer. We have created an example csv from the same `Acrobot-v1` environment, and have placed it under the `tutorials` folder.

Here are the first couple of lines from it so you can get a grip of what to expect - 

| action | all_action_probabilities | episode_id | episode_name | reward | transition_number | state_feature_0 | state_feature_1 | state_feature_2 | state_feature_3 | state_feature_4 | state_feature_5 
|---|---|---|---|---|---|---|---|---|---|---|---------------------------------------------------------------------------|
|0|[0.4159157,0.23191088,0.35217342]|0|acrobot|-1|0|0.996893843|0.078757007|0.997566524|0.069721088|-0.078539907|-0.072449002 |
|1|[0.46244532,0.22402011,0.31353462]|0|acrobot|-1|1|0.997643051|0.068617369|0.999777604|0.021088905|-0.022653483|-0.40743716|
|0|[0.4961428,0.21575058,0.2881066]|0|acrobot|-1|2|0.997613067|0.069051922|0.996147629|-0.087692077|0.023128103|-0.662019594|
|0|[0.49341106,0.22363988,0.28294897]|0|acrobot|-1|3|0.997141344|0.075558854|0.972780655|-0.231727853|0.035575821|-0.771402023|


### The Preset

In [14]:
tf.reset_default_graph() # just to clean things up; only needed for the tutorial

#########
# Agent #
#########
agent_params = DDQNBCQAgentParameters()
agent_params.network_wrappers['main'].batch_size = 128
agent_params.algorithm.num_steps_between_copying_online_weights_to_target = TrainingSteps(100)
agent_params.algorithm.discount = 0.99

# to jump start the agent's q values, and speed things up, we'll initialize the last Dense layer
# with something in the order of the discounted reward of a random policy
agent_params.network_wrappers['main'].heads_parameters = \
[QHeadParameters(output_bias_initializer=tf.constant_initializer(-100))]

# NN configuration
agent_params.network_wrappers['main'].learning_rate = 0.0001
agent_params.network_wrappers['main'].replace_mse_with_huber_loss = False

# ER - we'll be needing an episodic replay buffer for off-policy evaluation
agent_params.memory = EpisodicExperienceReplayParameters()

# E-Greedy schedule - there is no exploration in Batch RL. Disabling E-Greedy. 
agent_params.exploration.epsilon_schedule = LinearSchedule(initial_value=0, final_value=0, decay_steps=1)
agent_params.exploration.evaluation_epsilon = 0

# can use either a kNN or a NN based model for predicting which actions not to max over in the bellman equation
agent_params.algorithm.action_drop_method_parameters = KNNParameters()


DATATSET_PATH = 'acrobot_dataset.csv'
agent_params.memory = EpisodicExperienceReplayParameters()
agent_params.memory.load_memory_from_file_path = CsvDataset(DATATSET_PATH, is_episodic = True)

spaces = SpacesDefinition(state=StateSpace({'observation': VectorObservationSpace(shape=6)}),
                          goal=None,
                          action=DiscreteActionSpace(3),
                          reward=RewardSpace(1))

graph_manager = BatchRLGraphManager(agent_params=agent_params,
                                    env_params=None,
                                    spaces_definition=spaces,
                                    schedule_params=schedule_params,
                                    vis_params=VisualizationParameters(dump_signals_to_csv_every_x_episodes=1),
                                    reward_model_num_epochs=30,
                                    train_to_eval_ratio=0.4)
graph_manager.create_graph(task_parameters)
graph_manager.improve()


Creating graph - name: BatchRLGraphManager
Creating agent - name: agent
Loading a replay buffer from a CSV file. CSV file path: acrobot_dataset.csv
Progress: (30/30) Time: 4.43 sec 100%|##########|  
Training a regression model for estimating MDP rewards
Training Batch RL Models - Epoch: 0 Reward Model Loss: 78.00557143860902 
Training Batch RL Models - Epoch: 1 Reward Model Loss: 76.14748372554084 
Training Batch RL Models - Epoch: 2 Reward Model Loss: 73.58515795910306 
Training Batch RL Models - Epoch: 3 Reward Model Loss: 70.04382876116614 
Training Batch RL Models - Epoch: 4 Reward Model Loss: 65.50568371718522 
Training Batch RL Models - Epoch: 5 Reward Model Loss: 59.98599805447101 
Training Batch RL Models - Epoch: 6 Reward Model Loss: 53.793562782002006 
Training Batch RL Models - Epoch: 7 Reward Model Loss: 46.68347980979582 
Training Batch RL Models - Epoch: 8 Reward Model Loss: 39.663072881851605 
Training Batch RL Models - Epoch: 9 Reward Model Loss: 32.85732336419847 
Tra

INFO:tensorflow:./checkpoint/12_Step-0.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Checkpoint - Saving in path: ['./checkpoint/12_Step-0.ckpt'] 
Off-Policy Evaluation - Epoch: 13 IPS: -0.8652212601989943 DM: -7.195096969604492 DR: -3.770010014715164 WIS: -99.34293820733853 Sequential-DR: -99.68674557258258 
INFO:tensorflow:./checkpoint/13_Step-0.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Checkpoint - Saving in path: ['./checkpoint/13_Step-0.ckpt'] 
Off-Policy Evaluation - Epoch: 14 IPS: -0.8552962526076159 DM: -7.0609235763549805 DR: -3.637347618944837 WIS: -99.34292020325171 Sequential-DR: -99.36788101930414 
INFO:tensorflow:./checkpoint/14_Step-0.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Checkpoint - Saving in path: ['./checkpoint/14_Step-0.ckpt'] 
Off-Policy Evaluation - Epoch: 15 IPS: -0.8550920400128919 DM: -7.037744522094727 DR: -3.5705762026017904 WIS: -99.34289964763634 Sequential-DR: -99.39322828693764 
INFO:tensorflo

Checkpoint - Saving in path: ['./checkpoint/32_Step-0.ckpt'] 
Off-Policy Evaluation - Epoch: 33 IPS: -0.8635449200897655 DM: -7.114758014678955 DR: -3.4296058551686635 WIS: -98.90262838543394 Sequential-DR: -98.94610606486238 
INFO:tensorflow:./checkpoint/33_Step-0.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Checkpoint - Saving in path: ['./checkpoint/33_Step-0.ckpt'] 
Off-Policy Evaluation - Epoch: 34 IPS: -0.8689539109625974 DM: -7.171296119689941 DR: -3.471189229778747 WIS: -70.68976961491867 Sequential-DR: -98.92406395664024 
INFO:tensorflow:./checkpoint/34_Step-0.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Checkpoint - Saving in path: ['./checkpoint/34_Step-0.ckpt'] 
Off-Policy Evaluation - Epoch: 35 IPS: -0.8662159961660267 DM: -7.111026763916016 DR: -3.420302761995904 WIS: -70.79465550714923 Sequential-DR: -98.88323666461667 
INFO:tensorflow:./checkpoint/35_Step-0.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Checkpoint - Sa

Checkpoint - Saving in path: ['./checkpoint/52_Step-0.ckpt'] 
Off-Policy Evaluation - Epoch: 53 IPS: -0.8780532824978656 DM: -7.2137041091918945 DR: -3.482081599568536 WIS: -60.4205887667032 Sequential-DR: -98.5194581801334 
INFO:tensorflow:./checkpoint/53_Step-0.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Checkpoint - Saving in path: ['./checkpoint/53_Step-0.ckpt'] 
Off-Policy Evaluation - Epoch: 54 IPS: -0.8746697325133086 DM: -7.161681175231934 DR: -3.4114603867542543 WIS: -59.93982510521378 Sequential-DR: -98.49612932343157 
INFO:tensorflow:./checkpoint/54_Step-0.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Checkpoint - Saving in path: ['./checkpoint/54_Step-0.ckpt'] 
Off-Policy Evaluation - Epoch: 55 IPS: -0.8871427151454876 DM: -7.280055522918701 DR: -3.7198387522576373 WIS: -60.45529014574257 Sequential-DR: -98.47112489875087 
INFO:tensorflow:./checkpoint/55_Step-0.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Checkpoint - Sa

Checkpoint - Saving in path: ['./checkpoint/72_Step-0.ckpt'] 
Off-Policy Evaluation - Epoch: 73 IPS: -0.8941575009409074 DM: -7.2786712646484375 DR: -3.624156239603007 WIS: -60.57517165127425 Sequential-DR: -98.26024898438226 
INFO:tensorflow:./checkpoint/73_Step-0.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Checkpoint - Saving in path: ['./checkpoint/73_Step-0.ckpt'] 
Off-Policy Evaluation - Epoch: 74 IPS: -0.8951906869910556 DM: -7.327068328857422 DR: -3.6496746168939205 WIS: -60.58618687243636 Sequential-DR: -98.00244764281342 
INFO:tensorflow:./checkpoint/74_Step-0.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Checkpoint - Saving in path: ['./checkpoint/74_Step-0.ckpt'] 
Off-Policy Evaluation - Epoch: 75 IPS: -0.9000552194956029 DM: -7.364983081817627 DR: -3.7942245437502526 WIS: -60.62454434174176 Sequential-DR: -98.04676973396579 
INFO:tensorflow:./checkpoint/75_Step-0.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Checkpoint - 

Off-Policy Evaluation - Epoch: 93 IPS: -0.9096578218760405 DM: -7.200601577758789 DR: -4.236998416672068 WIS: -60.6800998629592 Sequential-DR: -95.33862089430488 
INFO:tensorflow:./checkpoint/93_Step-0.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Checkpoint - Saving in path: ['./checkpoint/93_Step-0.ckpt'] 
Off-Policy Evaluation - Epoch: 94 IPS: -0.9196709075397006 DM: -7.403517723083496 DR: -4.565778293176623 WIS: -60.662864183974946 Sequential-DR: -95.99971155364031 
INFO:tensorflow:./checkpoint/94_Step-0.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Checkpoint - Saving in path: ['./checkpoint/94_Step-0.ckpt'] 
Off-Policy Evaluation - Epoch: 95 IPS: -0.9118234316095112 DM: -7.315603733062744 DR: -4.312389435007319 WIS: -60.68975604842943 Sequential-DR: -96.5881746843874 
INFO:tensorflow:./checkpoint/95_Step-0.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Checkpoint - Saving in path: ['./checkpoint/95_Step-0.ckpt'] 
Off-Policy Evalua

### Model Selection with OPE
Running the above preset will train an agent based on the experience in the csv dataset. Note that now we are finally simulating the real scenario with Batch Reinforcement Learning, where we train and evaluate solely based on the recorded dataset. Coach uses the same dataset (after splitting it, obviously) for both training and evaluation. 

Now that we have ran this preset, we have 100 agents to choose from (one is saved after every training epoch), and we would have to decide which one we choose for deployment (either for running another round of experience collection and training, or for final deployment, meaning going into production).  

Openning the experiment csv in Dashboard and displaying the OPE signals, we can now choose a checkpoint file for deployment on the end-node. Here is an example run, where we show the `Weighted Importance Sampling` and `Sequential Doubly Robust` OPEs. 
</br>
![Model Selection](BatchRL/img/model_selection.png "Model Selection using OPE") 

Based on this plot we would probably have chosen a checkpoint from around Epoch 85 (but this is often more of an art than science). From here, if we are not satisfied with the agent's performance we can iteratively continue with data collection, training a new agent (maybe based on a combination of all the data collected so far), and deployment. 
